コンテンツ（内容）ベースフィルタリング：　
- コンテンツ(内容)の類似度を算出し、高い値のものをレコメンド
- https://qiita.com/haminiku/items/f5008a57a870e0188f63

形態素：　
- 言語が意味を持つ最小単位

形態素解析：　
- 文章を形態素に分割すること（例：「私/は/料理/し/ます」

TF-IDF：　
- 単語重要度の評価指標
- 2つの指標TF, IDFの積
- TF（Term Frequency）：　単語の出現頻度
- IDF（Inverse Document Frequency）：　逆文書頻度（単語のレア度）
- 単語の出現頻度が高いほど、またレアなほど値が大きい


In [2]:
# 形態素解析して名詞抽出、頻出名詞数/全名詞数を特徴ベクトル化（TFIDF）
# -*- coding: utf-8 -*-
from __future__ import absolute_import, unicode_literals
from collections import defaultdict
from math import sqrt
import re
import requests
from bs4 import BeautifulSoup
from janome.tokenizer import Tokenizer
import nltk


class TFIDF(object):
    _t = None

    @classmethod
    def gen(cls, text, enable_one_char=False):
        """
        Get TF-IDF
        :param text: str
        :rtype :list[list[str, float]]
        """
        _text = cls.filter(text)
        return cls.analysis(_text, enable_one_char=enable_one_char)

    @classmethod
    def gen_web(cls, url, enable_one_char=False):
        """
        Get TF-IDF from url
        :param url: str
        :rtype: list[list[str, float]]
        """
        # HTTP GET
        response = requests.get(url)

        # filter HTTP Tag
        soup = BeautifulSoup(response.text, "lxml")
        text = soup.title.name + soup.get_text()
        return cls.gen(text, enable_one_char=enable_one_char)

    @classmethod
    def similarity(cls, tfidf1, tfidf2):
        """
        Get TF-IDF and Cosine Similarity
        cosθ = A・B/|A||B|
        :param tfidf1: list[list[str, float]]
        :param tfidf2: list[list[str, float]]
        :rtype : float
        """
        tfidf2_dict = {key: value for key, value in tfidf2}

        ab = 0  # A・B
        for key, value in tfidf1:
            value2 = tfidf2_dict.get(key)
            if value2:
                ab += float(value * value2)

        # |A| and |B|
        a = sqrt(sum([v ** 2 for k, v in tfidf1]))
        b = sqrt(sum([v ** 2 for k, v in tfidf2]))

        return float(ab / (a * b))

    @classmethod
    def some_similarity(cls, base_url, data):
        """
        :param base_url: str
        :param data: list[lost[str, str]]
        :rtype : list[lost[str, str, float]]
        """
        base_tfidf = cls.gen_web(base_url)
        return [[title, url, cls.similarity(base_tfidf, cls.gen_web(url))] for title, url in data]

    @classmethod
    def analysis(cls, text, enable_one_char):
        """
        Calc TF-IDF
        textを形態素解析して名詞の数を返却(Morphological Analysis)
        :param text: str
        :rtype : dict{str: int}
        """
        result = defaultdict(int)
        result2 = {}
        count = 0
        t = cls._get_tokenizer()

        # 形態素解析
        for token in t.tokenize(text):
            if '名詞' not in token.part_of_speech:
                continue
            count += 1

            if '非自立' in token.part_of_speech:
                continue

            if '接尾' in token.part_of_speech:
                continue

            if '数' in token.part_of_speech:
                continue

            if not enable_one_char:
                if len(token.surface) == 1:
                    continue

            result[token.surface] += 1
            result2[token.surface] = token

        # TF-IDF計算
        result3 = []
        for key in result:
            result3.append([key, result[key]])

        result3.sort(key=lambda x: x[1], reverse=True)
        result4 = []
        for r in result3[:100]:
            # print r[0], float(float(r[1])/float(count)), result2[r[0]]
            result4.append([str(r[0]), float(float(r[1])/float(count))])
        return result4

    @classmethod
    def filter(cls, text):
        """
        textをフィルターしてノイズを排除する
        :param text: str
        :rtype : str
        """
        # アルファベットと半角英数と改行とタブを排除
        text = re.sub(r'[a-zA-Z0-9¥"¥.¥,¥@]+', '', text)
        text = re.sub(r'[!"“#$%&()\*\+\-\.,\/:;<=>?@\[\\\]^_`{|}~]', '', text)
        text = re.sub(r'[\n|\r|\t|年|月|日]', '', text)

        # 日本語以外の文字を排除(韓国語とか中国語とかヘブライ語とか)
        jp_chartype_tokenizer = nltk.RegexpTokenizer(u'([ぁ-んー]+|[ァ-ンー]+|[\u4e00-\u9FFF]+|[ぁ-んァ-ンー\u4e00-\u9FFF]+)')
        text = "".join(jp_chartype_tokenizer.tokenize(text))
        return text

    @classmethod
    def _get_tokenizer(cls):
        if TFIDF._t is not None:
            return TFIDF._t
        TFIDF._t = Tokenizer()
        return TFIDF._t

In [17]:
# 文章からTFIDFを出力(Get TF-IDF from text)
#from simple_tfidf_japanese.tfidf import TFIDF
text = "肉フェスNIIGATAで肉三昧の夜ごはん❤︎ステーキハウスあづまさんの雪室熟成新潟県産牛ステーキおいしい*\(^o^)/*お塩でもワサビでもぴったり！"
tfidf1 = TFIDF.gen(text, enable_one_char=1)
for key, value in tfidf1:
     print(key, value)

肉 0.09523809523809523
ステーキ 0.09523809523809523
フェス 0.047619047619047616
三昧 0.047619047619047616
夜 0.047619047619047616
ごはん 0.047619047619047616
ハウス 0.047619047619047616
あづま 0.047619047619047616
雪 0.047619047619047616
熟成 0.047619047619047616
新潟 0.047619047619047616
牛 0.047619047619047616
お 0.047619047619047616
塩 0.047619047619047616
ワサビ 0.047619047619047616


In [73]:
# 12の文章からTFIDFを算出、最初の文章(t[0]）とその他の11の文章との類似度算出
import numpy as np
# 文章定義
t = [0] * 12
t[0] = "スーパーマリオ 3Dワールド：　シリーズ最新作は、アクション要素がさらにパワーアップ。壁を上ったりひっかき攻撃を行える“ネコマリオ”が登場。さらに、最大で4人のマルチプレイもできる。"
t[1] = "マリオゴルフ ファミリーツアー： 　土管にボールが入るとワープするなどの仕掛けがある全7コースに挑戦。使用クラブをスロットで決めてラウンドしたり、ニアピンを競うモードなどもあるぞ。"
t[2] = "スーパードンキーコング2 ディクシー＆ディディー：あのきれいなグラフィックが画期的だった『スーパードンキーコング』が、パワーアップして再登場。今回も仲間との協力プレーで先のステージを目指せ。"
t[3] = "スーパードンキーコング：任天堂の次世代機、ウルトラ64の開発ツールで制作されたアクションゲーム。ドンキーコングとディディーコングが、40にもおよぶステージを冒険するのだ。"
t[4] = "スーパーマリオブラザーズ3：みんな知ってる、あの『スーパーマリオ』が帰ってきた。新しいアイテムや敵キャラ、そしてさまざまなシカケ。すべてが新しい『マリオ』なのだ。"
t[5] = "大乱闘スマッシュブラザーズ SPECIAL：多彩なアクションで相手を吹っ飛ばし、勝敗を決める人気シリーズ最新作。総勢75体のファイターが参戦し、ステージやアイテムなども大幅アップ。オンライン対戦にも対応。"
t[6] = "大乱闘スマッシュブラザーズX：マリオやスネークなど、さまざまなキャラクターを操作して、最大４人で対戦しよう。ふたりで協力してプレイするモードを追加。オンライン対戦も可能だぞ。"
t[7] = "ドラゴンクエストビルダーズ2 破壊神シドーとからっぽの島：『ドラゴンクエスト?　悪霊の神々』のその後の世界が舞台。“からっぽ島”を中心に、仲間集めやモノ作りをしながら、多彩な島を開拓していく。最大4人でのマルチプレイも可能。"
t[8] = "ゼルダの伝説 トワイライトプリンセス：リンクを操作し、?トワイライト?という影の領域とハイラルを行き来して物語を進めよう。ブーメランなどのアイテムを駆使して、さまざまな仕掛けを解くのだ。"
t[9] = "ヨッシー ウールワールド：毛糸で表現された世界を舞台に、編みぐるみのヨッシーが大冒険。“おなじみモード”と“エンジョイモード”の2種類を収録するほか、シリーズ初のふたり同時プレイにも対応。"
t[10] = "星のカービィ 夢の泉の物語：ゲームボーイで人気だったアクションゲームがファミコンで登場する。今回はスライディングキックや水鉄砲などの、新アクションが加わっているぞ。"
t[11] = "戦国BASARA2：シリーズ第2弾。戦国武将を操り、一騎当千の戦いを楽しもう。身体能力を一時的に上昇させる新要素、?戦極ドライブ?によって、より高い爽快感が味わえる。"

# 最初の文章のTFIDF算出
ti[0] = TFIDF.gen(t[0], enable_one_char=1)
for n in range(12):
    # TFIDF算出
    ti[n] = TFIDF.gen(t[n], enable_one_char=1)
    # 類似度計算
    s = TFIDF.similarity(ti[0], ti[n])
    print("t[{}] : {}".format(n, round(s, 2)))

t[0] : 1.0
t[1] : 0.0
t[2] : 0.15
t[3] : 0.0
t[4] : 0.15
t[5] : 0.12
t[6] : 0.06
t[7] : 0.11
t[8] : 0.0
t[9] : 0.0
t[10] : 0.18
t[11] : 0.07


In [10]:
# URLからWeb文章を取得し、TFIDFを出力(Get TF-IDF from Web)
#url = "https://ja.wikipedia.org/wiki/%E6%B7%A1%E8%B7%AF%E3%83%93%E3%83%BC%E3%83%95"
url = "https://ja.wikipedia.org/wiki/%E4%BB%8A%E3%81%84%E3%81%8F%E3%82%88%E3%83%BB%E3%81%8F%E3%82%8B%E3%82%88"
tfidf2 = TFIDF.gen_web(url)
for key, value in tfidf2:
     print(key, value)

編集 0.014240506329113924
京都 0.011867088607594937
漫才 0.011867088607594937
出典 0.011075949367088608
出演 0.011075949367088608
ラジオ 0.010284810126582278
お笑い 0.00949367088607595
大賞 0.00949367088607595
閲覧 0.00870253164556962
いくよ 0.007911392405063292
吉本興業 0.007120253164556962
テレビ 0.007120253164556962
芸人 0.006329113924050633
記事 0.005537974683544304
コンビ 0.005537974683544304
大阪 0.005537974683544304
ページ 0.005537974683544304
花王 0.004746835443037975
名人 0.004746835443037975
所属 0.004746835443037975
ネタ 0.004746835443037975
放送 0.004746835443037975
出身 0.003955696202531646
舞台 0.003955696202531646
上方 0.003955696202531646
表示 0.003955696202531646
リンク 0.003955696202531646
胃がん 0.003955696202531646
追加 0.003955696202531646
必要 0.0031645569620253164
女流 0.0031645569620253164
師匠 0.0031645569620253164
学校 0.0031645569620253164
レギュラー 0.0031645569620253164
番組 0.0031645569620253164
映画 0.0031645569620253164
復帰 0.0031645569620253164
最後 0.0031645569620253164
相方 0.0031645569620253164
参列 0.0031645569620253164
人物 0.00316455696202

In [74]:
# URLからWeb文書を取得しTFIDF計算 x 2セットの類似度比較
# -*- coding: utf-8 -*-
from __future__ import absolute_import, unicode_literals
#from simple_tfidf_japanese.tfidf import TFIDF

ikuyo_wikipedia_url = 'https://ja.wikipedia.org/wiki/%E4%BB%8A%E3%81%84%E3%81%8F%E3%82%88%E3%83%BB%E3%81%8F%E3%82%8B%E3%82%88'
korochiki_wikipedia_url = 'https://ja.wikipedia.org/wiki/%E3%82%B3%E3%83%AD%E3%82%B3%E3%83%AD%E3%83%81%E3%82%AD%E3%83%81%E3%82%AD%E3%83%9A%E3%83%83%E3%83%91%E3%83%BC%E3%82%BA'
ikuyo_tfidf = TFIDF.gen_web(ikuyo_wikipedia_url)
chiki_tfidf = TFIDF.gen_web(korochiki_wikipedia_url)

# 類似度計算
print(TFIDF.similarity(ikuyo_tfidf, chiki_tfidf))

0.5172215004666876


In [ ]:
# -> 類似度が高いものをレコメンドすれば良い。